# Data Processing script v2 for the SWEML v2.0
This .ipynb script uses python module for processing predownloaded NASA ASO observations by Water Year, locating nearest SNOTEL sites, connecting SNOTEL obs with ASO obs, and add geospatial features to the ML training/testing/hindcast dataframes.

In [5]:
os.chdir('/uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/SWEMLv2.0/Dataprocessing')

In [1]:
import os
import numpy as np
HOME = os.getcwd()

#If you get a proj.db error below, run the following and put the following into the terminal
import pyproj
# Get the PROJ data directory
proj_data_dir = pyproj.datadir.get_data_dir()
proj_db_path = proj_data_dir + "/proj.db"
os.environ['PROJ_LIB'] =pyproj.datadir.get_data_dir()
os.environ['PROJ_LIB']
#set multiprocessing limits
CPUS = len(os.sched_getaffinity(0))
CPUS = int((CPUS/2)-2)

#set home to the head of the SWEMLv2.0 directory
HOME = os.chdir('..')
HOME = os.getcwd()

#Add your module here
# from utils.ASOget import ASODataProcessing_v2
import utils.get_InSitu_obs as get_InSitu_obs
import utils.GeoDF as GeoDF 
import utils.Obs_to_DF as Obs_to_DF 
import utils.get_VIIRS_SCA as get_VIIRS_SCA
import utils.get_Precip as get_Precip
import utils.get_Seasonality as get_Seasonality
import utils.vegetation_processer as vegpro
import utils.sturm_processer as stpro
from utils import ASOget



#make SWEMLv2.0 modeling domain for western USA
# WY_list = ['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024'] #'2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024' - Trying the first bit to prove concept, then can expand
# WY_list = np.arange(2013,2025)
WY_list = [2024]
output_res = 750 #desired spatial resolution in meters (m)
threshold = 10
print(f"The current session is using {WY_list} years, {output_res}m resolution, and {CPUS} CPUs")

AWS access keys loaded
The current session is using [2024] years, 750m resolution, and 30 CPUs


In [2]:
# Inputs for fetching ASO data for a region
short_name = 'ASO_50M_SWE'
directory = "Raw_ASO_Data"

#Get ASO data, sometime sites will give error and break code, most times you can just rerun it using the data_processor sections below (e.g., comment out other parts
for WY in WY_list:
    #Convert ASO tifs to parquet
    print(f"Converting ASO images for WY: {WY}")
    folder_name = f"{WY}/{directory}"
    data_processor = ASOget.ASODataProcessing_v2() #note, 2019-5-1, 2019-06-11 seems to be bad, manually removed from SW region
    data_processor.convert_tiff_to_parquet_multiprocess(folder_name, output_res, WY) 

Converting ASO images for WY: 2013
Converting .tif to parquet
Converting 6 ASO tif files to parquet


  0%|          | 0/6 [00:00<?, ?it/s]

Checking to make sure all files successfully converted...


  0%|          | 0/6 [00:00<?, ?it/s]

Converting ASO images for WY: 2014
Converting .tif to parquet
Converting 18 ASO tif files to parquet


...

  0%|          | 0/45 [00:00<?, ?it/s]

Converting ASO images for WY: 2023
Converting .tif to parquet
Converting 82 ASO tif files to parquet


  0%|          | 0/82 [00:00<?, ?it/s]

Checking to make sure all files successfully converted...


  0%|          | 0/82 [00:00<?, ?it/s]

## Get Snotel and CDEC in situ observations
- Ideas - add nearest sites elevation, distance from cell, then can bypass sites with bad data. 

In [ ]:
# Only needed once. Other spatial resolutions can use the same data
#Get in situ observations

#make a list of dates to align with the ASO observations (they go as early as Jan-29 and as far out as the July-17)
years = np.arange(2013,2025,1)
start_month_day = '10-01'
end_month_day = '08-31'

# observations 
get_InSitu_obs.Get_Monitoring_Data_Threaded_Updated(years, start_month_day, end_month_day, WY = True)

#combine years
get_InSitu_obs.combine_dfs(years)

## Code for generating ML dataframe using nearest in situ monitoring sites

In [ ]:
# GeoDF used to create a dataframe for ML model development. Its function is to connect in situ observations to gridded locations
for WY in WY_list:
    path = f"{HOME}/data/ASO/{WY}/{output_res}M_SWE_parquet"

    if os.path.isdir(path) == True:
        print(WY)
        #load snotel meta location data, use haversive function
        GeoDF.fetch_snotel_sites_for_cellids(WY, output_res) # Using known up to date sites

        # Get geophysical attributes for each site, need to see how to add output resolution
        gdf = GeoDF.GeoSpatial(WY, output_res)

        #use geodataframe with lat/long meta of all sites to determine slope, aspect, and elevation
        metadf = GeoDF.extract_terrain_data_threaded(gdf, WY, output_res)
    else:
        print(f"No ASO data for {WY}")




2024
Loading all Geospatial prediction/observation files and concatenating into one dataframe


  0%|          | 0/70 [00:00<?, ?it/s]

Identifying unique sites to create geophysical information dataframe
converting to geodataframe
Processing snotel geometry
Calculating haversine distance for 129061 locations to in situ OBS, and saving cell-obs relationships in dictionary


  0%|          | 0/129061 [00:00<?, ?it/s]

Saving nearest SNOTEL in 2024 for each cell id in a pkl file
Loading geospatial data for 2024
Converting to geodataframe
Calculating dataframe bounding box
-123 34 -104 44
Retrieving Copernicus 90m DEM tiles


  0%|          | 0/215 [00:00<?, ?it/s]

There are 215 tiles in the region
Determining Grid Cell Spatial Features


  0%|          | 0/129061 [00:00<?, ?it/s]

2024_750M_39.545_-106.08 does not have copernicus DEM data, manual input


## Connect Snotel to each ASO obs


In [ ]:
#Rerun

#Connect nearest snotel observations with ASO data, makes a parquet file for each date  -  test to see if this works - need to just load the SNOTEL file, not collect them as in the function
for WY in WY_list:
    path = f"{HOME}/data/ASO/{WY}/{output_res}M_SWE_parquet"

    if os.path.isdir(path) == True:
        print(WY)
        dates = []
        manual = False
        Obs_to_DF.Nearest_Snotel_2_obs_MultiProcess(WY, output_res, manual, dates) 
    else:
        print(f"No ASO data for {WY}")


2013
Connecting site observations with nearest monitoring network obs
Loading observations
Loading 250M resolution grids for 2013 region
Processing datetime component of SNOTEL observation dataframe
Loading 6 processed ASO observations for the 2013 at 250M resolution
There are 6 aso dates in snotel obs
There are 0 missing snotel obs
Connecting 6 timesteps of observations for 2013


100%|██████████| 6/6 [00:00<00:00,  9.30it/s]

Site processing complete, adding observtional data to 20130403 df...


  0%|          | 0/23806 [00:00<?, ?it/s]

Site processing complete, adding observtional data to 20130429 df...


...

Site processing complete, adding observtional data to 20190609 df...


  0%|          | 0/13495 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



/uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/SWEMLv2.0/data/TrainingDFs/2019/250M_Resolution/Obsdf/USCOCM_20190407_ObsDF.parquet
/uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/SWEMLv2.0/data/TrainingDFs/2019/250M_Resolution/Obsdf/USCOCM_20190610_ObsDF.parquet
/uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/SWEMLv2.0/data/TrainingDFs/2019/250M_Resolution/Obsdf/USCAKN_20190417_ObsDF.parquet
/uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/SWEMLv2.0/data/TrainingDFs/2019/250M_Resolution/Obsdf/USCAKN_20190611_ObsDF.parquet
/uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/SWEMLv2.0/data/TrainingDFs/2019/250M_Resolution/Obsdf/USCAMB_20190329_ObsDF.parquet
/uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/SWEMLv2.0/data/TrainingDFs/2019/250M_Resolution/Obsdf/USCOGT_20190408_ObsDF.parquet
/uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/SWEMLv2.0/data/TrainingDFs/2019/250M_Resolution/Obsdf/USCOGT_20190609_ObsDF.parquet
/uufs/chpc.utah.edu/common/home/ci

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [3]:
#Connect cell ids with ASO obs and snotel obs to geospatial features
for WY in WY_list:
    #path = f"{HOME}/SWEMLv2.0/data/ASO/{region}/{output_res}M_SWE_parquet"
    path = f"{HOME}/data/ASO/{WY}/{output_res}M_SWE_parquet"
    if os.path.isdir(path) == True:
        print(WY)
        GeoDF.add_geospatial_threaded(WY, output_res)
    else:
        print(f"No ASO data for {region}")

2020
Loading goeospatial meta data for grids in 2020
Loading all available processed ASO observations for the 2020 at 250M resolution
Concatenating 14 with geospatial data...


  0%|          | 0/14 [00:00<?, ?it/s]

Job complete for connecting obs with geospatial data, the files can be found in /uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/SWEMLv2.0/data/TrainingDFs/2020/250M_Resolution/GeoObsDFs
2021
Loading goeospatial meta data for grids in 2021
Loading all available processed ASO observations for the 2021 at 250M resolution
Concatenating 19 with geospatial data...


  0%|          | 0/19 [00:00<?, ?it/s]

Job complete for connecting obs with geospatial data, the files can be found in /uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/SWEMLv2.0/data/TrainingDFs/2021/250M_Resolution/GeoObsDFs
2022
Loading goeospatial meta data for grids in 2022
Loading all available processed ASO observations for the 2022 at 250M resolution
Concatenating 45 with geospatial data...


  0%|          | 0/45 [00:00<?, ?it/s]

Job complete for connecting obs with geospatial data, the files can be found in /uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/SWEMLv2.0/data/TrainingDFs/2022/250M_Resolution/GeoObsDFs
2023
Loading goeospatial meta data for grids in 2023
Loading all available processed ASO observations for the 2023 at 250M resolution
Concatenating 82 with geospatial data...


  0%|          | 0/82 [00:00<?, ?it/s]

Job complete for connecting obs with geospatial data, the files can be found in /uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/SWEMLv2.0/data/TrainingDFs/2023/250M_Resolution/GeoObsDFs


## Get NASA VIIRS fraction snow covered area for each location 

* Make sure the code grabs all dates for each region, may have to run multiple times
* run until "No granules found for DATE, requesting data from NSIDC..." no longer occurs


In [ ]:
get_VIIRS_SCA.get_VIIRS_from_AWS()

In [7]:
#check to see if the VIIRS data is available locally, if not, get from CIROH AWS - I think all of this data is for the incorrect year...
#get_VIIRS_SCA.get_VIIRS_from_AWS()

#Connect VIIRS data to dataframes
for WY in WY_list:
    path = f"{HOME}/data/ASO/{WY}/{output_res}M_SWE_parquet"
    if os.path.isdir(path) == True:
        print(WY)
        get_VIIRS_SCA.augment_SCA_multiprocessing(WY, output_res, threshold)
    else:
        print(f"No ASO data for {WY}")


2013
Getting VIIRS fsca values for 6 timesteps of observations for 2013
20130403, 20130429, 20130525, 20130503, 20130601, 20130608, 





dataprocessing VIIRS for 20130608 complete...
dataprocessing VIIRS for 20130429 complete...
dataprocessing VIIRS for 20130525 complete...
dataprocessing VIIRS for 20130403 complete...
dataprocessing VIIRS for 20130503 complete...
dataprocessing VIIRS for 20130601 complete...
Job complete for connecting VIIRS fsca to sites/dates, files can be found in /uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/SWEMLv2.0/data/TrainingDFs/2013/250M_Resolution/VIIRSGeoObsDFs/10_fSCA_Thresh
2014
Getting VIIRS fsca values for 18 timesteps of observations for 2014
20140324, 20140406, 20140414, 20140423, 20140512, 20140429, 20140503, 20140420, 20140323, 
20140428, 
20140407, 20140511, 20140502, 
20140517, 20140527, 20140531, 20140605, 
20140320, 













dataprocessing VIIRS for 20140320 complete...
dataprocessing VIIRS for 20140324 complete...
dataprocessing

## Add seasonality metrics to the dataframe

In [2]:
for WY in WY_list:
    #process snotel sites to make "snow hydrograph features" to determine above/below average WY conditions
    get_Seasonality.seasonal_snotel()


    #get the Day of season metric for each dataframe
    get_Seasonality.add_Seasonality(WY, output_res, threshold)

Adding Day of Season, seasonal nearest monitoring site averages, and  nearest monitoring site anomoly to averages to all 2024 dataframes...


  0%|          | 0/70 [00:00<?, ?it/s]

100%|██████████| 9284/9284 [00:04<00:00, 2288.69it/s]


## Use Sturm's snow classification as features within model framework

Using the originally created env, it looks like the rasterio package does not contain the correct ECS driver. Trying to address this with conda install conda-forge::rasterio in my SWEML_310 env from the shell in CHPC

In [3]:
#download sturm data
stpro.get_Sturm_data()

for WY in WY_list:
    print(WY)
    input_directory = f"{HOME}/data/TrainingDFs/{WY}/{output_res}M_Resolution/Seasonality_VIIRSGeoObsDFs/{threshold}_fSCA_Thresh"
    sturm_file = f"{HOME}/data/SnowClassification/SnowClass_NA_300m_10.0arcsec_2021_v01.0.tif" #https://nsidc.org/data/nsidc-0768/versions/1
    output_directory = f"{HOME}/data/TrainingDFs/{WY}/{output_res}M_Resolution/Sturm_Seasonality_VIIRSGeoObsDFs/{threshold}_fSCA_Thresh"
    
    # Create output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)
    
    stpro.process_sturm_data_for_files(input_directory, sturm_file, output_directory)

sturm data already downloaded
2024
Sturm file bounds: BoundingBox(left=-180.0, bottom=0.0, right=-10.0, top=90.0)
Adding Sturm snow classificaction to ASO gridcells


  0%|          | 0/70 [00:00<?, ?it/s]

## Add vegetation data to the dataframe from the North American land Cover Management Systemoutput_path

This script needs to be multiprocessed, too slow

In [4]:
#get data
url = "http://www.cread_parquetiles/atlas_layers/1_terrestrial_ecosystems/1_01_0_land_cover_2020_30m/usa_land_cover_2020v2_30m_tif.zip"
output_path = f"{HOME}/data/LandCover/"
file = "usa_land_cover_2020v2_30m_tif.zip" 
vegpro.get_data(url, output_path, file)
#unzip the file is not already done
#vegpro.unzip_LC_data(output_path, file)
#output = 1000 

for WY in WY_list:
    print(WY)
    input_directory = f"{HOME}/data/TrainingDFs/{WY}/{output_res}M_Resolution/Sturm_Seasonality_VIIRSGeoObsDFs/{threshold}_fSCA_Thresh"
    vegetation_file = f"{HOME}/data/LandCover/usa_land_cover_2020v2_30m_tif/USA_NALCMS_landcover_2020v2_30m/data/USA_NALCMS_landcover_2020v2_30m.tif"
    output_directory = f"{HOME}/data/TrainingDFs/{WY}/{output_res}M_Resolution/Vegetation_Sturm_Seasonality_VIIRSGeoObsDFs/{threshold}_fSCA_Thresh"
    
    # Create output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)
    
    vegpro.process_vegetation_data_for_files(input_directory, vegetation_file, output_directory)

Landcover data already downloaded
2024
Adding Vegetation criteria to ASO grids


  0%|          | 0/70 [00:00<?, ?it/s]

## Get Daymet, NLDAS, gridMET Precipitation for each cell

In [5]:
datasets = [# 'Daymet',
            #  'NLDAS',
             'gridMET'
           ]

for dataset in datasets:
    for WY in WY_list:
        path = f"{HOME}/../../SWEMLdata/ASO/{WY}/{output_res}M_SWE_parquet"

        if os.path.isdir(path) == True:
            print(WY)
            # get_Precip.get_precip(WY, output_res, threshold, dataset)
        else:
            print(f"No ASO data for {WY}, {path}")

        #Connect precipitation to processed DFs
        get_Precip.Make_Precip_DF(WY, output_res, threshold, dataset)

2024
Adding precipitation features to ML dataframe for 2024.
Connecting precipitation to ASO observations for 2024 on 2024-04-04 at DoloresConnecting precipitation to ASO observations for 2024 on 2024-05-20 at Taylor
Connecting precipitation to ASO observations for 2024 on 2024-05-22 at RoaringForkConnecting precipitation to ASO observations for 2024 on 2024-03-14 at SpragueConnecting precipitation to ASO observations for 2024 on 2024-05-08 at ConejosConnecting precipitation to ASO observations for 2024 on 2024-03-21 at WindyGapConnecting precipitation to ASO observations for 2024 on 2024-03-19 at UintaMountainsConnecting precipitation to ASO observations for 2024 on 2024-04-14 at WindyGapConnecting precipitation to ASO observations for 2024 on 2024-05-30 at WindyGap
Connecting precipitation to ASO observations for 2024 on 2024-02-10 at Yuba
Connecting precipitation to ASO observations for 2024 on 2024-05-27 at YampaRiverConnecting precipitation to ASO observations for 2024 on 2024-04-

## Get AORC Precip for each cell

In [6]:
dataset = 'AORC'
for WY in WY_list:
    path = f"{HOME}/../../SWEMLdata/ASO/{WY}/{output_res}M_SWE_parquet"

    if os.path.isdir(path) == True:
        print(WY)
        # get_Precip.get_aorc_precip(WY, output_res, threshold)
    else:
        print(f"No ASO data for {WY}, {path}")

    #Connect precipitation to processed DFs
    get_Precip.Make_Precip_DF(WY, output_res, threshold, dataset)

2024
Adding precipitation features to ML dataframe for 2024.
Connecting precipitation to ASO observations for 2024 on 2024-05-08 at ConejosConnecting precipitation to ASO observations for 2024 on 2024-05-20 at TaylorConnecting precipitation to ASO observations for 2024 on 2024-04-28 at KaweahConnecting precipitation to ASO observations for 2024 on 2024-04-04 at DoloresConnecting precipitation to ASO observations for 2024 on 2024-05-20 at KaweahConnecting precipitation to ASO observations for 2024 on 2024-02-11 at KaweahConnecting precipitation to ASO observations for 2024 on 2024-06-07 at CarsonConnecting precipitation to ASO observations for 2024 on 2024-04-03 at KaweahConnecting precipitation to ASO observations for 2024 on 2024-05-16 at CarsonConnecting precipitation to ASO observations for 2024 on 2024-02-22 at CarsonConnecting precipitation to ASO observations for 2024 on 2024-04-17 at CarsonConnecting precipitation to ASO observations for 2024 on 2024-03-21 at Carson
Connecting p

In [9]:
import pandas as pd
test = pd.read_parquet(f'{HOME}/data/TrainingDFs/2024/750M_Resolution/AORCgridMETNLDASDaymet_Vegetation_Sturm_Seasonality_VIIRSGeoObsDFs/10_fSCA_Thresh/AORC_gridMET_NLDAS_Vegetation_Sturm_Season_VIIRS_GeoObsdf_BigThompson_20240421.parquet')
test.head()

,cell_id,Date,cen_lat,cen_lon,Elevation_m,Slope_Deg,Aspect_Deg,swe_m,ns_1,ns_2,...,ns_2_anomoly,ns_3_anomoly,ns_4_anomoly,ns_5_anomoly,ns_6_anomoly,sturm_value,vegetation_value,NLDAS,gridMET,AORC
0,2024_750M_40.541_-105.501,2024-04-21,40.541,-105.501,3195,26,66,0.063879,22.86,41.402,...,0.915730,0.932115,1.068555,0.938848,0.985094,2,10,29.6,31.5,40.9
1,2024_750M_40.534_-105.501,2024-04-21,40.534,-105.501,3058,17,182,0.146466,22.86,41.402,...,0.915730,0.932115,1.068555,0.938848,0.985094,2,10,29.6,31.5,40.4
2,2024_750M_40.534_-105.494,2024-04-21,40.534,-105.494,3011,39,76,0.035186,22.86,41.402,...,0.915730,0.932115,1.068555,0.938848,0.985094,2,10,17.2,26.8,39.5
3,2024_750M_40.534_-105.487,2024-04-21,40.534,-105.487,3073,33,78,0.014969,22.86,41.402,...,0.915730,0.932115,1.068555,0.938848,0.985094,2,10,17.2,26.8,38.3
4,2024_750M_40.527_-105.507,2024-04-21,40.527,-105.507,3364,20,216,0.264527,22.86,47.498,...,0.932115,0.915730,1.068555,0.938848,0.985094,2,8,29.6,31.5,40.8


## Next steps
* Explore why errors in precip sites above
* add in situ obs - seasonality based on the historical neareste x monitoring stations - like a historical average to-date swe value unit hydrograph based on the day of year? This will include a historical time of year of normal swe value and a swe value of year compared to normal
* albedo metric


In [ ]:
import pandas as pd

HOME = os.path.expanduser('~')
region = 'Southwest'
output_res = '300'

dfpath = f"{HOME}/SWEMLv2.0/data/TrainingDFs/{region}/{output_res}M_Resolution"

SWmeta = pd.read_parquet(f"{dfpath}/{region}_metadata.parquet")

import UpdateDataFrame

#need to update the topographic features for every dataframe
output_res = '300'
training_cats = ['Obsdf']
fSCA = '' #'20_fSCA_Thresh'


for training_cat in training_cats:
    print(training_cat)

    for region in region_list:
        print(region)
        dfpath = f"{HOME}/SWEMLv2.0/data/TrainingDFs/{region}/{output_res}M_Resolution"
        #file to be used to updated training DF
        updatefile = pd.read_parquet(f"{dfpath}/{region}_metadata.parquet")


        #Update Dataframe
        UpdateDataFrame.updateTrainingDF(region, output_res, training_cat, fSCA, updatefile)

trainfile = pd.read_parquet(f"{dfpath}/{training_cat}/{fSCA}/Sturm_Season_Precip_VIIRS_GeoObsDF_20150406.parquet")

import matplotlib.pyplot as plt
import geopandas as gpd

from mpl_toolkits.axes_grid1 import make_axes_locatable

def SpatialAnalysis(EvalDF):
    #Convert to a geopandas DF
    Pred_Geo = gpd.GeoDataFrame(EvalDF, geometry = gpd.points_from_xy(EvalDF.cen_lon, EvalDF.cen_lat))

    Pred_Geo.plot(column='Elevation_m',
                  legend=False,
                )
    
SpatialAnalysis(trainfile)

In [ ]:
import pandas as pd

In [ ]:
test = pd.read_parquet(f"{HOME}/data/TrainingDFs/2023/500M_Resolution/NLDASDaymet_Vegetation_Sturm_Seasonality_VIIRSGeoObsDFs/10_fSCA_Thresh/NLDAS_PrecipDaymet_Vegetation_Sturm_Season_VIIRS_GeoObsdf_BoulderCreek_20230509.parquet")
test.head()

In [ ]:
years = np.arange(2013,2024)
# years = [2020,2021,2022]
cols = ['cen_lat', 'cen_lon', 'Elevation_m', 'Slope_Deg',
       'Aspect_Deg', 'swe_m', 'ns_1', 'ns_2', 'ns_3', 'ns_4', 'ns_5', 'ns_6',
       'VIIRS_SCA', 'hasSnow', 'DOS', 'WY_week', 'ns_1_week_mean',
       'ns_2_week_mean', 'ns_3_week_mean', 'ns_4_week_mean', 'ns_5_week_mean',
       'ns_6_week_mean', 'ns_1_anomoly', 'ns_2_anomoly', 'ns_3_anomoly',
       'ns_4_anomoly', 'ns_5_anomoly', 'ns_6_anomoly', 
        # 'sturm_value',       'vegetation_value', 'Daymet', 'NLDAS'
       ]

for year in years:
    
    files = [filename for filename in os.listdir(f"{HOME}/data/TrainingDFs/{year}/1000M_Resolution/NLDASDaymet_Vegetation_Sturm_Seasonality_VIIRSGeoObsDFs/10_fSCA_Thresh/")]
    # files = [filename for filename in os.listdir(f"{HOME}/../Johnson/SWEMLv2.0/data/TrainingDFs/{year}/1000M_Resolution/Sturm_Seasonality_PrecipVIIRSGeoObsDFs/10_fSCA_Thresh/")]

    for file in files:
        test = pd.read_parquet(f"{HOME}/data/TrainingDFs/{year}/1000M_Resolution/NLDASDaymet_Vegetation_Sturm_Seasonality_VIIRSGeoObsDFs/10_fSCA_Thresh/{file}")
    # test = test.drop(columns='Date')
    # test.reset_index().drop(columns='index').head()
    # test.columns
        for column in cols:
            # print(len(test[test[column]<1]))
            # if len(test[test[column]<1]) > 500:
                # print(column)
            if np.isnan(test[column]).any():
                print('nan',file, column)
            if np.isinf(test[column]).any():
                print('inf',file,column)

# test.drop(columns='Elevation_m')[test.drop(columns='Elevation_m')>250].shape

In [ ]:
filename = f"{HOME}/data/TrainingDFs/2020/500M_Resolution/Seasonality_VIIRSGeoObsDFs/10_fSCA_Thresh/Season_VIIRS_GeoObsdf_Merced_20200604.parquet"
test = pd.read_parquet(filename)
test[test['ns_6_anomoly']==np.inf]
12/test.iloc[185]['ns_1_week_mean']

In [ ]:
pdf2019 = pd.read_parquet(f'{HOME}/Predictions/NLDASDaymet_Vegetation_Sturm_Seasonality_VIIRSGeoObsDFs/PDF_Test_WY2019_1000M_Resolution.parquet')
# pdf.head()
pdf2020 = pd.read_parquet(f'{HOME}/Predictions/NLDASDaymet_Vegetation_Sturm_Seasonality_VIIRSGeoObsDFs/PDF_Test_WY2020_1000M_Resolution.parquet')
pdf2019
pdf2021 = pd.read_parquet(f'{HOME}/Predictions/NLDASDaymet_Vegetation_Sturm_Seasonality_VIIRSGeoObsDFs/PDF_Test_WY2021_1000M_Resolution.parquet')
pdf2021

In [ ]:
daymet2014 = pd.read_parquet(f'{HOME}/Predictions/Daymet_Vegetation_Sturm_Seasonality_VIIRSGeoObsDFs/PDF_Test_WY2014_1000M_Resolution.parquet')
daymet2014